In [1]:
import numpy as np
import pandas as pd

#datasetleri yüklüyoruz
#df_test=pd.read_excel("/content/drive/MyDrive/kolektif_proje/TurkishTweets.xlsx")
df_test = pd.read_csv("/content/drive/MyDrive/kolektif_proje/magaza_yorumlari.csv", encoding = "utf-16")
#df_test = pd.read_csv("/content/drive/MyDrive/kolektif_2/train.csv",index_col=[0],encoding="windows-1252");

In [2]:
list(df_test)

['Görüş', 'Durum']

In [3]:
import torch
# cuda varsa gpu yoksa cpu kullanmak için cihazı ayarlayan kod hücresi
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: Tesla V100-SXM2-16GB


In [4]:
df_test

,Görüş,Durum
0,Ürün gayet güzel boyutları dört kişilik aile ...,Olumlu
1,Ürünü kullandım gayet memnunum,Olumlu
2,Hızlı kargo,Olumlu
3,8 yıl önce lg 42 lm serisi almıştım ve tv den ...,Olumsuz
4,ürün fena değil ancak gövdenin plastik olması ...,Olumlu
...,...,...
8486,Ürün güzel mavi olanı tercih ettim bana göre s...,Olumlu
8487,kesinlikle kullandığım en kötü rimel kirpikler...,Olumsuz
8488,Fiyatı uygun gelebilir evet lakin malzeme Yani...,Olumsuz
8489,indirimdeyken aldik cok memnunuz digerleriyle...,Olumlu


In [5]:
encoded_labels = {'Olumlu': 0, 'Olumsuz': 1}
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['Durum'] = label_encoder.fit_transform(df_test['Durum'])

In [6]:
#test veri seti için zero-shot a sokulmak üzere metin ve etiketlerin alınması ardından ilk 1000inin alınması
comments = df_test.Görüş.values

labels_test = df_test.Durum.values

comments = comments[:1000]
labels_test = labels_test[:1000]

In [7]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig
import pickle
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F

accuracy_list= []
model_predicts = {}
#her 5 model için iterasyon kuruldu  'ytu-ce-cosmos/turkish-small-bert-uncased', 'ytu-ce-cosmos/turkish-mini-bert-uncased','ytu-ce-cosmos/turkish-medium-bert-uncased',
models= ['ytu-ce-cosmos/turkish-small-bert-uncased', 'ytu-ce-cosmos/turkish-mini-bert-uncased','ytu-ce-cosmos/turkish-medium-bert-uncased']
for model_name in models:
    name= model_name.split("/")[-1]
    print(name)
    #tokenizer ve model burada tanımlanıyor
    tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=True)
    model = AutoModel.from_pretrained(model_name, num_labels=2)
    hidden_layer = model.config.num_hidden_layers
    print(f"Model has {hidden_layer} hidden layer.")


    #commentsin içerisindeki her satır için vektörler çıkartılıp cosine uzaklık predict_listin içerisine yazılıyor ardından bu liste model_predictsin içerisine yazılıyor. Ayrıca Accuracyler de Accuracy_list e yazılıyor


    labels = ["bu bir olumlu","bu bir olumsuz"]

    prediction_dict = {}

    predict_list = []
    for i,comment in enumerate(comments):
        sentence= comment
            # run inputs through model and mean-pool over the sequence
        # dimension to get sequence-level representations
        inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                            return_tensors='pt',
                                            pad_to_max_length=True)

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        try:
            output = model(input_ids, attention_mask=attention_mask,output_hidden_states=True)[0]
            sentence_rep = output[:1].mean(dim=1)
            label_reps = output[1:].mean(dim=1)

            # now find the labels with the highest cosine similarities to
            # the sentence
            similarities = F.cosine_similarity(sentence_rep, label_reps)
            #closest en yakın olan labelı önce yazıyor closest[0] en yakını vermiş oluyor
            closest = similarities.argsort(descending=True)
            predict_list.append(closest[0])
        except:
            predict_list.append(closest[0])
    prediction_dict["pooled_layer"]= predict_list

    for layer_index in range(model.config.num_hidden_layers+1):
        print(layer_index)
        predict_list = []
        for comment in comments:
            sentence = comment
            # run inputs through model and mean-pool over the sequence
            # dimension to get sequence-level representations
            inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                                return_tensors='pt',
                                                pad_to_max_length=True)
            try:
                input_ids = inputs['input_ids']
                attention_mask = inputs['attention_mask']
                output = model(input_ids, attention_mask=attention_mask,output_hidden_states=True)[2][layer_index]
                sentence_rep = output[:1].mean(dim=1)
                label_reps = output[1:].mean(dim=1)


                # now find the labels with the highest cosine similarities to
                # the sentence
                similarities = F.cosine_similarity(sentence_rep, label_reps)
                #closest en yakın olan labelı önce yazıyor closest[0] en yakını vermiş oluyor
                closest = similarities.argsort(descending=True)
                predict_list.append(closest[0])
            except:
                predict_list.append(closest[0])

        prediction_dict[layer_index] = predict_list

    with open(f'/content/drive/MyDrive/magaza_prediction_dict_{name}_etiket2.pickle', 'wb') as handle:
        pickle.dump(prediction_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

bert-base-turkish-cased


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Model has 12 hidden layer.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

0
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
import pickle
with open('/content/drive/MyDrive/magaza_prediction_dict_123.pickle', 'wb') as handle:
    pickle.dump(prediction_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
predict_list